In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# 1. 读取 CSV 文件
csv_file_path = 'combined_all.csv'
column_names = ['chr', 'start','end','gene_name','strand']
df = pd.read_csv(csv_file_path, names=column_names, header=None)
# # print(df.head(1))
# df = df.dropna(subset=['exp_score'])
# df['exp_score'] = df['exp_score'].astype(str)
# df['exp_score'] = df['exp_score'].str.strip()
# df = df[df['exp_score'] != '']
# # df = df[df['exp_score'].notna() & (df['exp_score'] != '')]
# print(df.head(10))


In [ ]:
# 2. 连接 MySQL 数据库
# 假设使用 PyMySQL 作为驱动程序
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# # 3. 过滤掉第三列为空的行
# df = df.dropna(subset=['exp_score'])

# # 3. 添加其他固定字段
# df['exp_type'] = 'CRISPR CasRx'
# df['role'] = ''
# df['cell_line'] = 'NICH460'



# 6. 将数据存入数据库
Session = sessionmaker(bind=engine)
session = Session()

df.to_sql('merge', con=engine, if_exists='append', index=False)

session.commit()
session.close()


In [ ]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('test.csv')

# 找到重复项
duplicates = df[df.duplicated('name', keep=False)]

# 只保留每组重复项中的一个
unique_duplicates = duplicates.drop_duplicates(subset='name', keep='first')

# 将结果保存到新的CSV文件
unique_duplicates.to_csv('unique_duplicates.csv', index=False)

print("已将有重复项的行保存到 'unique_duplicates.csv' 文件中。")

# 将csv转换为bed格式

In [ ]:
import pandas as pd

# 读取CSV文件
column_names = ['gene_id', 'chromosome', 'strand', 'start', 'end']
df = pd.read_csv('casRX_gtf.csv',names=column_names)

# 将start列减1以符合BED格式（BED格式的起始位置是从0开始）
df['start'] = df['start'] - 1
# 选择所需列的顺序，并添加必要的列
df['chromosome'] = 'chr' + df['chromosome'].astype(str)
bed_df = df[['chromosome', 'start', 'end', 'gene_id', 'strand']]
bed_df.loc[:, 'score'] = 0  # BED文件中的第五列通常是score，这里设为0
bed_df = bed_df[['chromosome', 'start', 'end', 'gene_id', 'score', 'strand']]

# 将DataFrame保存为BED格式
bed_df.to_csv('transcripts.bed', sep='\t', header=False, index=False)


### 旧版本genename 和 id 映射

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 读取CSV文件
csv_file_path = './map/sup.csv'
column_names = ['gene', 'ensemble', 'gene_name']
df = pd.read_csv(csv_file_path, names=column_names, header=None)

# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')
table_name = 'map_temp2'

# 将DataFrame存入MySQL数据库
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print("数据已成功存入MySQL数据库中。")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
# 读取CSV文件
csv_file_path = './map/map-lnc9.csv'
column_names = ['lib_id', 'Lncbook_id']
df = pd.read_csv(csv_file_path, names=column_names, header=None)
print(df.head(10))

# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
table_name = 'merge'

# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 将DataFrame存入MySQL数据库
with engine.connect() as connection:
    for index, row in df.iterrows():
        # ens-map
        # lib_id = row['lib_id']
        # old_name = row['old_name']

        # non-map
        lib_id = row['lib_id']
        Lncbook_id = row['Lncbook_id']
        if pd.notna(lib_id):  # 仅在 old_name 非空时更新
            query = text(f'''
                UPDATE {table_name}
                SET Lncbook_id = '{Lncbook_id}' 
                WHERE lib_id = '{lib_id}';
            ''')
            connection.execute(query, {'lib_id': lib_id, 'Lncbook_id': Lncbook_id})
    
    connection.commit()

print("数据已成功更新到MySQL数据库中。")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# 读取CSV文件的前两列
csv_file_path = './map/merged_site2.csv'
df = pd.read_csv(csv_file_path, usecols=[0, 1], names=['name', 'id'], header=None)

# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
table_name = 'merge'

# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 将DataFrame存入MySQL数据库
with engine.connect() as connection:
    for index, row in df.iterrows():
        name = row['name']
        ensembl_id = row['id']
        if pd.notna(ensembl_id):  # 仅在 ensembl_id 非空时更新
            # 查找匹配的 gene_name
            query = text(f'''
                SELECT gene_name FROM {table_name}
                WHERE gene_name LIKE CONCAT('%', :name, '%');
            ''')
            result = connection.execute(query, {'name': name}).fetchone()
            if result:
                # 更新 ensembl_id
                update_query = text(f'''
                    UPDATE {table_name}
                    SET ensembl_id = :ensembl_id
                    WHERE gene_name LIKE CONCAT('%', :name, '%');
                ''')
                connection.execute(update_query, {'ensembl_id': ensembl_id, 'name': name})
    connection.commit()  # 提交事务

print("数据已成功更新到MySQL数据库中。")

In [ ]:
import pandas as pd

# 读取CSV文件
csv_file_path = './map/update.csv'
df = pd.read_csv(csv_file_path, names=['ensembl_id', 'gene_name', 'gene', 'ens_id'], header=None)

# 遍历每一行，检查条件并打印
for index, row in df.iterrows():
    ensembl_id = row['ensembl_id']
    ens_id = row['ens_id']
    if pd.notna(ensembl_id) and ensembl_id != ens_id:
        print(f"ensembl_id: {ensembl_id}, gene_name: {row['gene_name']}, gene: {row['gene']}, ens_id: {ens_id}")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 读取文件
file_path = './map/non-coding_RNA.txt'
df = pd.read_csv(file_path, sep='\t')

# 选择指定的列
columns_to_keep = ['hgnc_id', 'symbol', 'name', 'alias_symbol', 'prev_symbol', 'entrez_id', 'ensembl_gene_id', 'pubmed_id', 'lncipedia']
df = df[columns_to_keep]
print(df.head(10))
# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
table_name = 'map_name'

# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 将DataFrame存入MySQL数据库
df.to_sql(table_name, engine, if_exists='replace', index=False)

print("数据已成功导入到MySQL数据库中。")

## 添加 文献原始的数据

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 读取文件
file_path = './map/site-pair38.bed'
df = pd.read_csv(file_path, sep='\t', header=None, names=['chr', 'start', 'end', 'Lit_id', 'dot', 'strand'])

# 选择指定的列并处理数据
df['start'] = df['start'] + 1
columns_to_keep = ['Lit_id', 'chr', 'strand', 'start', 'end']
df = df[columns_to_keep]

# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
table_name = 'crispr_lit'

# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 将DataFrame存入MySQL数据库
df.to_sql(table_name, engine, if_exists='append', index=False)

print("数据已成功导入到MySQL数据库中。")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# 读取文件
file_path = './map/result.txt'
df = pd.read_csv(file_path, sep='\t', header=None, names=['NCBI_id','UID',"Go_annotation"])


# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'
table_name = 'esslnc'

# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 更新数据库中的数据
# with engine.connect() as connection:
#     for index, row in df.iterrows():
#         lit_id = row['Lit_id']
#         ensembl_id = row['ensembl_id']
#         gene_name = row['gene_name']
        
#         # 更新语句
#         update_query = text(f"""
#             UPDATE {table_name}
#             SET ensembl_id = :ensembl_id, gene_name = :gene_name
#             WHERE Lit_id = :lit_id
#         """)
        
#         # 执行更新
#         connection.execute(update_query, {"ensembl_id": ensembl_id, "gene_name": gene_name, "lit_id": lit_id})

#     connection.commit()    
# print("数据已成功更新到MySQL数据库中。")
# max_id = 0
# 更新ID列
# 获取需要更新的行
    # result = connection.execute(text(f"SELECT * FROM {table_name} WHERE Organism = 'Mouse'"))
    # rows = result.fetchall()
    
    # # 执行更新
    # for i, row in enumerate(rows, start=1):
    #     new_id = max_id + i
    #     new_id_str = f"ELM{new_id:06d}"
    #     update_query = text(f"""
    #         UPDATE {table_name}
    #         SET ID = :new_id
    #         WHERE Organism = 'Mouse' AND ID IS NULL
    #         LIMIT 1
    #     """)
    #     connection.execute(update_query, {"new_id": new_id_str})
with engine.connect() as connection:
    
    for index, rows in df.iterrows():
        Go_annotation = rows['Go_annotation']
        UID = rows['UID']
        if pd.notna(Go_annotation):
            update_query = text(f"""
                UPDATE {table_name}
                SET Go_annotation = :Go_annotation
                WHERE UID = :UID
            """)
            connection.execute(update_query, {"Go_annotation": Go_annotation, "UID": UID})
    connection.commit()

print("数据已成功更新到MySQL数据库中。")

## 更新数据库数据

In [2]:
# 创建数据连接
import pandas as pd
from sqlalchemy import create_engine, text

# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'


# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')



In [5]:
#更新数据库内容

table_name = 'trans'

# 读取文件
# file_path = './map/score.csv'
# df = pd.read_csv(file_path, sep=',')
# print(df.head(10))
# df = pd.read_csv(file_path, sep=',', header=None, names=['UID','chr','start','end','strand','source','Lncbook_id','Lncbook_trans_id'])

with engine.connect() as connection:
    query = "SELECT UID, variation_id FROM lncrna_variant_mapping"
    df = pd.read_sql(query, connection)
    grouped_df = df.groupby('UID')['variation_id'].apply(lambda x: ';'.join(x.astype(str))).reset_index()

    # 保存为 CSV 文件
    grouped_df.to_csv('disease_related.csv', index=False)

print("CSV 文件已生成：output.csv")



CSV 文件已生成：output.csv


In [ ]:
# 获取转录本信息
import pandas as pd

# 读取 UID.csv 文件的第一列信息
uid_file_path = './map/UID.csv'
uid_df = pd.read_csv(uid_file_path,names=['gene_id','UID'])
gene_ids = uid_df['gene_id'].tolist()

# 读取 GTF 文件并筛选出包含这些信息的行
gtf_file_path = './map/gencode.v19.long_noncoding_RNAs.gtf'
filtered_lines = []

with open(gtf_file_path, 'r') as gtf_file:
    for line in gtf_file:
        if any(gene_id in line for gene_id in gene_ids):
            filtered_lines.append(line)

# 将筛选后的行保存到新的文件中
output_file_path = './map/deletion.gtf'
with open(output_file_path, 'w') as output_file:
    output_file.writelines(filtered_lines)

print("筛选后的行已成功保存到新的 GTF 文件中。")

In [ ]:
import pandas as pd

# 读取 GTF 文件
gtf_file_path = './map/deletion.gtf'
gtf_df = pd.read_csv(gtf_file_path, sep='\t', header=None, comment='#',
                     names=['chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])

# 提取 gene_id 和 transcript_id
def extract_attributes(attribute):
    attributes = attribute.split(';')
    gene_id = ''
    transcript_id = ''
    gene_name = ''
    for attr in attributes:
        if 'gene_id' in attr:
            gene_id = attr.split('"')[1]
        elif 'transcript_id' in attr:
            transcript_id = attr.split('"')[1]
        elif 'gene_name' in attr:
            gene_name = attr.split('"')[1]
    return gene_id, transcript_id, gene_name

gtf_df['gene_id'], gtf_df['transcript_id'], gtf_df['gene_name'] = zip(*gtf_df['attribute'].apply(extract_attributes))

# # 创建 BED 文件的第四列
# gtf_df['bed_name'] = gtf_df.apply(lambda row: f"{row['gene_id']},{row['transcript_id']},{row['feature']},{row['gene_name']} ", axis=1)

# # 创建 BED 文件
# bed_df = gtf_df[['chrom', 'start', 'end', 'bed_name', 'score', 'strand']]

# # 保存为 BED 文件
bed_file_path = './map/deletion_map.bed'

gtf_df[['transcript_id','gene_name']].to_csv(bed_file_path, sep='\t', header=False, index=False)

print("GTF 文件已成功转换为 BED 文件。")

In [ ]:
import csv

input_file = './map/unfa.txt'
output_file = './map/unfa1.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile, delimiter='\t')
    writer = csv.writer(outfile, delimiter='\t')
    
    for row in reader:
        # 删除前五列
        new_row = row[5:]
        writer.writerow(new_row)

print(f"前五列已从 {input_file} 中删除，并保存到 {output_file}")

In [ ]:
# 序列截取
import re

# 读取txt文件中的位置信息
def read_exon_positions(txt_file):
    exon_positions = {}
    id_map = {}
    with open(txt_file, 'r', encoding='utf-8') as file:
        next(file)  # 跳过第一行
        for line in file:
            parts = line.strip().split('\t')
            # print(parts)
            if len(parts) > 1:
                target_id = parts[3].strip()
                transcript_id = parts[4].strip()
                exon_pos = parts[-1]
                exon_positions[transcript_id] = exon_pos.split(',')
                id_map[transcript_id] = target_id
    return exon_positions, id_map

# 读取fa文件中的序列
def read_fasta(fasta_file):
    sequences = {}
    with open(fasta_file, 'r', encoding='utf-8') as file:
        seq_id = ""
        seq = ""
        strand = "+"
        start_pos = 0
        end_pos = 0
        for line in file:
            if line.startswith(">ELH"):
                if seq_id:
                    sequences[seq_id] = (seq, start_pos , end_pos, strand)
                seq_id = line.strip().split()[0][1:]  # 去掉 '>' 并获取ID
                seq = ""
            elif line.startswith(">chromosome"):
                parts = line.strip().split(':')
                start_pos = int(parts[3])
                end_pos = int(parts[4])
                strand = "-" if parts[-1] == ("-1") else "+"
                
            else:
                seq += line.strip()
        if seq_id and seq:
            sequences[seq_id] = (seq, start_pos, end_pos, strand)
    return sequences

# 根据位置信息截取并拼接序列
def extract_and_concatenate_sequence(seq, positions, start_pos, end_pos, strand):
    new_seq = ""
    # print(strand)
    for pos in positions:
        start, end = map(int, pos.split('-'))
        if start < start_pos or end > end_pos:
            # print(f"位置信息超出范围",start,end)
            continue
        # print(start,end)
        if strand == "+":
            # 计算相对位置
            relative_start = start - start_pos
            relative_end = end - start_pos + 1
            fragment = seq[relative_start:relative_end]
            # print(relative_start,relative_end)
            new_seq += fragment
        else:
            # 计算相对位置，从尾部开始
            relative_start = end_pos - end
            relative_end = end_pos - start + 1
            fragment = seq[relative_start:relative_end]
            new_seq = fragment + new_seq
            # print(fragment)
            
    return new_seq

def gen_fa(sequences, exon_positions, id_map, outfile):
     with open(outfile, 'w', encoding='utf-8') as file:
        for transcript_id, positions in exon_positions.items():
            uid= id_map[transcript_id]
            if uid in sequences:
                seq, start_pos, end_pos, strand = sequences[uid]
            
                new_seq = extract_and_concatenate_sequence(seq, positions, start_pos, end_pos, strand)
                if new_seq == "":
                    print(f"位置信息不正确",transcript_id)
                    continue
                file.write(f">{transcript_id}\n")
                file.write(f"{new_seq}\n")
            else:
                print(f"未找到 {transcript_id} 的序列")
# 主函数
def main():
    txt_file = './map/unfa.txt'
    fasta_file = './map/seq.fa'
    outfile = './map/trans.fa'
    trans_id = 'URS0000D5BD64_9606_2'

    exon_positions,id_map = read_exon_positions(txt_file)

    sequences = read_fasta(fasta_file)

    for transcript_id, positions in exon_positions.items():
        uid= id_map[transcript_id]
        if uid in sequences:
            seq, start_pos, end_pos, strand = sequences[uid]
        
            new_seq = extract_and_concatenate_sequence(seq, positions, start_pos, end_pos, strand)
            if new_seq == "":
                print(transcript_id)
                continue
        else:
                print(f"未找到 {transcript_id} 的序列")
    # 生成fa文件
    # gen_fa(sequences, exon_positions, id_map, outfile)

    
    # test
    # positions = exon_positions[trans_id]
    # seq, start_pos, end_pos, strand = sequences[id_map[trans_id]]
    # print(f"序列ID",id_map[trans_id])

    # new_seq = extract_and_concatenate_sequence(seq, positions, start_pos, end_pos, strand)
    # print(new_seq)

    
    # print(f"拼接后的新序列:\n{new_seq}")
    # print(f"新序列长度: {len(new_seq)}")
    
    

if __name__ == "__main__":
    main()

In [ ]:
# 更新数据库的fa文件
# 创建数据连接
import pandas as pd
from sqlalchemy import create_engine, text

# 数据库连接信息
username = 'root'
password = 'auggie'
host = 'localhost'  # 通常是 'localhost'
port = 3306  # 默认 MySQL 端口号
database = 'dbess'


# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 解析trans.fa文件
def parse_fa_file(fa_file):
    sequences = {}
    with open(fa_file, 'r', encoding='utf-8') as file:
        transcript_id = ""
        seq = ""
        for line in file:
            if line.startswith(">"):
                if transcript_id and seq:
                    sequences[transcript_id] = seq
                transcript_id = line.strip()[1:]  # 去掉 '>'
                seq = ""
            else:
                seq += line.strip()
        if transcript_id and seq:
            sequences[transcript_id] = seq  # 添加最后一个序列
    return sequences

def update_database(fa_file, engine):
    # 解析fa文件
    sequences = parse_fa_file(fa_file)
    
    with engine.connect() as connection:
    
        for Lncbook_trans_id, sequence in sequences.items():
            update_query = text("""
                UPDATE trans
                SET FASTA = CONCAT('>', transcript_id, '|', Lncbook_trans_id, '<br/>', :sequence)
                WHERE Lncbook_trans_id = :Lncbook_trans_id
            """)
            connection.execute(update_query, {"sequence": sequence, "Lncbook_trans_id": Lncbook_trans_id})
        connection.commit()




fa_file = './map/outLncRNA.fa'
txt_file = './map/Non_trans.txt'
df = pd.read_csv(txt_file, sep='\t', header=0)
trans = set(df['NONCODE_TRANSCRIPT_ID'])
# print(trans)
with engine.connect() as connection:
    with open(fa_file, 'r', encoding='utf-8') as file:
        NONCODE_TRANSCRIPT_ID = ""
        seq = ""
        for line in file:
            if line.startswith(">"):
                if NONCODE_TRANSCRIPT_ID and seq and NONCODE_TRANSCRIPT_ID in trans:
                    update_query = text("""
                        UPDATE trans
                        SET FASTA = CONCAT('>', transcript_id, '<br/>', :sequence)
                        WHERE NONCODE_TRANSCRIPT_ID = :NONCODE_TRANSCRIPT_ID AND FASTA IS NULL
                    """)
                    connection.execute(update_query, {"sequence": seq, "NONCODE_TRANSCRIPT_ID": NONCODE_TRANSCRIPT_ID})
                NONCODE_TRANSCRIPT_ID = line.strip()[1:]
                seq = ""
            else:
                seq += line.strip()
        if NONCODE_TRANSCRIPT_ID and seq and NONCODE_TRANSCRIPT_ID in trans:
                update_query = text("""
                        UPDATE trans
                        SET FASTA = CONCAT('>', transcript_id, '<br/>', :sequence)
                        WHERE NONCODE_TRANSCRIPT_ID = :NONCODE_TRANSCRIPT_ID AND FASTA IS NULL
                    """)
                connection.execute(update_query, {"sequence": seq, "NONCODE_TRANSCRIPT_ID": NONCODE_TRANSCRIPT_ID})
    connection.commit()
    print("数据已成功更新到MySQL数据库中。")



In [ ]:
def parse_fa_file(fa_file):
    sequences = {}
    with open(fa_file, 'r', encoding='utf-8') as file:
        transcript_id = ""
        seq = ""
        for line in file:
            if line.startswith(">"):
                if transcript_id and seq:
                    sequences[transcript_id] = seq
                transcript_id = line.strip()[1:]  # 去掉 '>'
                seq = ""
            else:
                seq += line.strip()
        if transcript_id and seq:
            sequences[transcript_id] = seq  # 添加最后一个序列
    return sequences

fa_file = './map/LncBookv2_OnlyLnc.fa'
seq = parse_fa_file(fa_file)
print(seq)

In [ ]:
## 构建blast库
# 读取TXT文件内容
with open('./blast/lncrna/lncRNA2.txt', 'r', encoding='utf-8') as file:
    txt_content = file.read()

# 将 <br/> 替换为换行符
fasta_content = txt_content.replace('<br/>', '\n')



# 将结果保存为FASTA文件
with open('./blast/lncrna/lncRNA2.fasta', 'w', encoding='utf-8') as file:
    file.write(fasta_content)

print("转换完成，结果已保存为 output.fasta")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 读取数据文件
boxplot_data = pd.read_csv('boxplot_data.txt', sep='\t', header=None, names=['Value'])  # 全部数据
highlight_data = pd.read_csv('highlight.txt', sep='\t', header=None, names=['Value'])  # 需要标注的点

# 添加标记列，标记特殊点
boxplot_data['IsHighlight'] = boxplot_data['Value'].isin(highlight_data['Value'])

# 设置颜色映射
colors = boxplot_data['IsHighlight'].map({True: 'red', False: 'gray'})

# 绘制箱线图
plt.figure(figsize=(8, 6))
sns.boxplot(y='Value', data=boxplot_data, showfliers=False, width=0.3)

# # 绘制普通点
# sns.stripplot(y='Value', data=boxplot_data[~boxplot_data['IsHighlight']], 
#               color='gray', alpha=0.6, jitter=True)

# 绘制特殊点
sns.stripplot(y='Value', data=boxplot_data[boxplot_data['IsHighlight']], 
              color='red', alpha=0.8, size=8, label='Highlight Points')

# 设置标题和图例
plt.legend(title="Point Type", loc='upper right')
plt.title("Boxplot with Highlighted Points (No Category)", fontsize=14)
plt.ylabel("Value", fontsize=12)

plt.tight_layout()
plt.show()
